
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_configuration     |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |

In [3]:
%number_of_workers  2
%glue_version '3.0'

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.34 and you have 0.30 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Previous number of workers: 5
Setting new number of workers to: 2
Setting Glue version to: 3.0


## Default Import Statements

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

## Read Data in CSV From AWS S3

In [2]:
order_df = glueContext.create_dynamic_frame_from_options(connection_type= 's3',
                                                               connection_options={"paths": ['s3://s3_bucket/raw/directory/']},
                                                               format='csv', format_options = {"withHeader": True, "optimizePerformance": True})

## Display schema and first 5 records

In [3]:
order_df.printSchema()
order_df.toDF().show(5)

root
|-- order_id: string
|-- customer_id: string
|-- total_amount: string
|-- order_date: string

+--------+--------------------+------------+----------+
|order_id|         customer_id|total_amount|order_date|
+--------+--------------------+------------+----------+
|    1000|43fb5b29-3b6d-4a3...|      642.66|2022-08-24|
|     999|d510e386-4d2b-416...|     1324.86|2022-08-27|
|     998|a743267d-544d-464...|       76.37|2022-08-24|
|     997|e2ac237c-fae2-49e...|      981.05|2022-08-26|
|     996|ace8c4f9-5f10-497...|      750.84|2022-08-27|
+--------+--------------------+------------+----------+
only showing top 5 rows


## Remap Data Types to Correct Data Type

In [4]:
order_df_transformed =  ApplyMapping.apply(frame=order_df,
                                           mappings=[("order_id", "string", "order_id", "int"),
                                        ("customer_id", "string", "customer_id", "string"),
                                        ("total_amount", "string", "total_amount", "double"),
                                        ("order_date", "string", "order_date", "string")],
                                           transformation_ctx="order_df_transformation")
order_df_transformed.printSchema()

root
|-- order_id: int
|-- customer_id: string
|-- total_amount: double
|-- order_date: string


## Write To AWS S3 and Create Table in Glue Catalog

In [6]:
write_S3_parquet  = glueContext.getSink(
     path="s3://s3_bucket/processed/output_directory/",
     connection_type="s3",
     updateBehavior="UPDATE_IN_DATABASE",
     partitionKeys=["order_date"],
    compression= "gzip",
    enableUpdateCatalog=True,
    transformation_ctx="Write_S3_parquet")

write_S3_parquet.setCatalogInfo(catalogDatabase="customer", catalogTableName="order")
write_S3_parquet.setFormat("glueparquet")
write_S3_parquet.writeFrame(order_df_transformed)